In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

#pd.options.mode.chained_assignment = None - removes warnings. 

In [2]:
#Open sas dataset from brfss containing obesity information for education, income, age and ethnicity.
brfss_edudf = pd.read_sas('LLCP2016.XPT')



In [3]:
brfss_edudf.head(5)

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_MAM5021,_RFPAP33,_RFPSA21,_RFBLDS3,_COL10YR,_HFOB3YR,_FS5YR,_FOBTFS,_CRCREC,_AIDTST3
0,1.0,1.0,b'01072016',b'01',b'07',b'2016',1100.0,b'2016000001',2.016000e+09,1.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1.0,1.0,b'01112016',b'01',b'11',b'2016',1100.0,b'2016000002',2.016000e+09,1.0,...,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,2.0
2,1.0,1.0,b'01062016',b'01',b'06',b'2016',1100.0,b'2016000003',2.016000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,1.0,1.0,b'01082016',b'01',b'08',b'2016',1100.0,b'2016000004',2.016000e+09,1.0,...,NaN,NaN,1.0,2.0,1.0,2.0,NaN,2.0,1.0,9.0
4,1.0,1.0,b'01052016',b'01',b'05',b'2016',1100.0,b'2016000005',2.016000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [4]:
brfss_edudf["_STATE"].unique()

array([ 1.,  2.,  4.,  5.,  6.,  8.,  9., 10., 11., 12., 13., 15., 16.,
       17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29.,
       30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42.,
       44., 45., 46., 47., 48., 49., 50., 51., 53., 54., 55., 56., 66.,
       72., 78.])

In [5]:
# read fips to state reference data. 
fip_state = pd.read_csv('fips_to_state.csv')
fip_state.head()

,fips,state_abbr
0,1,AL
1,2,AK
2,4,AZ
3,5,AR
4,6,CA


In [6]:
#List column names for the data frame.
brfss_edudf.columns


Index(['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE',
       'SEQNO', '_PSU', 'CTELENM1',
       ...
       '_MAM5021', '_RFPAP33', '_RFPSA21', '_RFBLDS3', '_COL10YR', '_HFOB3YR',
       '_FS5YR', '_FOBTFS', '_CRCREC', '_AIDTST3'],
      dtype='object', length=275)

In [7]:
# Re-naming the _STATE to fips  and brfss dataframe for merging the FIPS DF. 
brfss_edudf = brfss_edudf.rename(columns={"_STATE":"fips"})
brfss_merge = pd.merge(brfss_edudf,fip_state,on="fips")
# Dropping the fips column as it is not required further
#**.drop(["fips"],axis=1,inplace=True)
brfss_merge.head(5)

,fips,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_RFPAP33,_RFPSA21,_RFBLDS3,_COL10YR,_HFOB3YR,_FS5YR,_FOBTFS,_CRCREC,_AIDTST3,state_abbr
0,1,1.0,b'01072016',b'01',b'07',b'2016',1100.0,b'2016000001',2.016000e+09,1.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,AL
1,1,1.0,b'01112016',b'01',b'11',b'2016',1100.0,b'2016000002',2.016000e+09,1.0,...,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,2.0,AL
2,1,1.0,b'01062016',b'01',b'06',b'2016',1100.0,b'2016000003',2.016000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AL
3,1,1.0,b'01082016',b'01',b'08',b'2016',1100.0,b'2016000004',2.016000e+09,1.0,...,NaN,1.0,2.0,1.0,2.0,NaN,2.0,1.0,9.0,AL
4,1,1.0,b'01052016',b'01',b'05',b'2016',1100.0,b'2016000005',2.016000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,AL


In [8]:
#create a new dataframe with the columns needed for education, income with year and state information included.
#fill in columns with data based on the 2016 codebook
#reference: https://www.cdc.gov/brfss/annual_data/2016/pdf/codebook16_llcp.pdf
brfss_edu_inc = brfss_merge[['state_abbr', 'IDATE' , '_BMI5CAT', '_RFBMI5', '_EDUCAG', 'INCOME2']]

brfss_edu_inc.head(5)


,state_abbr,IDATE,_BMI5CAT,_RFBMI5,_EDUCAG,INCOME2
0,AL,b'01072016',2.0,1.0,2.0,5.0
1,AL,b'01112016',3.0,2.0,2.0,7.0
2,AL,b'01062016',2.0,1.0,3.0,7.0
3,AL,b'01082016',3.0,2.0,4.0,7.0
4,AL,b'01052016',2.0,1.0,2.0,77.0


In [9]:
## options to correctly decode IDATE values to UTF-8 format.
pd.options.mode.chained_assignment = None

In [10]:
brfss_edu_inc["IDATE"] = brfss_edu_inc["IDATE"].str.decode("utf-8")
brfss_edu_inc["_BMI5CAT"] = brfss_edu_inc["_BMI5CAT"].replace({1:"Underweight", 2.0:"Normal Weight", 3.0:"Overweight", 4.0:"Obese", None:"Don't know/Refused/Missing"})
brfss_edu_inc["_RFBMI5"]= brfss_edu_inc["_RFBMI5"].replace({1.0:"No", 2.0:"Yes", 9.0:"Don’t know/Refused/Missing"})
brfss_edu_inc["_EDUCAG"] = brfss_edu_inc["_EDUCAG"].replace({1.0:"Did not graduate High School", 2.0:"Graduated High School", 
                                                           3.0:"Attended College or Technical School", 
                                                           4.0:"Graduated from College or Technical School", 
                                                           9.0: "Don’t know/Not sure/Missing"})

brfss_edu_inc["INCOME2"] = brfss_edu_inc["INCOME2"].replace({1.0:"Less than $10,000", 2.0:"$10,000 to less than $15,000",
                                                             3.0:"$15,000 to less than $20,000", 4.0:"$20,000 to less than $25,000", 
                                                             5.0:"$25,000 to less than $35,000", 6.0: "$35,000 to less than $50,000",
                                                             7.0:"$50,000 to less than $75,000",8.0:"$75,000 or more",
                                                             8.0:"$75,000 or more",77.0:"Don’t know/Not sure",99:"Refused",
                                                             None:"Not asked or Missing"
                                                            })
brfss_edu_inc.head(5)

,state_abbr,IDATE,_BMI5CAT,_RFBMI5,_EDUCAG,INCOME2
0,AL,01072016,Normal Weight,No,Graduated High School,"$25,000 to less than $35,000"
1,AL,01112016,Overweight,Yes,Graduated High School,"$50,000 to less than $75,000"
2,AL,01062016,Normal Weight,No,Attended College or Technical School,"$50,000 to less than $75,000"
3,AL,01082016,Overweight,Yes,Graduated from College or Technical School,"$50,000 to less than $75,000"
4,AL,01052016,Normal Weight,No,Graduated High School,Don’t know/Not sure


In [11]:
brfss_edu_inc= brfss_edu_inc.rename(columns={"state_abbr": "State", 
                              "IDATE": "Date", 
                              "_BMI5CAT": "BMI Category", 
                              "_RFBMI5": "BMI Over 25", 
                              "_EDUCAG": "Education_Level",
                              "_INCOME2": "Income_range"})
brfss_edu_inc.reset_index(inplace=True,drop=True)


In [15]:
brfss_edu_inc.tail(10)

,State,Date,BMI Category,BMI Over 25,Education_Level,INCOME2
482441,VI,12172016,Normal Weight,No,Graduated from College or Technical School,"Less than $10,000"
482442,VI,12112016,Normal Weight,No,Graduated from College or Technical School,"$20,000 to less than $25,000"
482443,VI,12242016,Overweight,Yes,Attended College or Technical School,"$15,000 to less than $20,000"
482444,VI,12092016,Normal Weight,No,Graduated High School,"$35,000 to less than $50,000"
482445,VI,12312016,Obese,Yes,Graduated from College or Technical School,"$35,000 to less than $50,000"
482446,VI,12312016,Overweight,Yes,Did not graduate High School,"Less than $10,000"
482447,VI,12192016,Normal Weight,No,Graduated High School,Refused
482448,VI,12092016,Obese,Yes,Attended College or Technical School,"$20,000 to less than $25,000"
482449,VI,12312016,Obese,Yes,Graduated High School,"$20,000 to less than $25,000"
482450,VI,12312016,Overweight,Yes,Did not graduate High School,"Less than $10,000"


In [13]:
#save to csv
brfss_edu_inc.to_csv("ObesityEduIncome_2016.csv",index=False)